In [ ]:
from clickhouse_driver import Client
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()

password = os.getenv('CLICKHOUSE_PASSWORD')

client = Client(
    host='localhost',
    port=9000,
    database='mma_stats_silver',
    user='default',
    password=password
)

fights_columns = [
    "Athlete_ID",
    "Opponent_ID",
    "Fight_Date",
    "Fight_Result"
]

fights_query = """SELECT Athlete_ID,
                         Opponent_ID,
                         Fight_Date,
                         Fight_Result 
                  FROM ufc_athletes_records"""

fights = pd.DataFrame(client.execute(fights_query), columns=fights_columns)

fights["Fight_Date"] = pd.to_datetime(fights["Fight_Date"])
fights["Delta_Time"] = (datetime.now() - fights["Fight_Date"]).dt.days / 365.25
fights["Delta_Time"] = round(fights["Delta_Time"], 0)

/tmp/ipykernel_3242/588528488.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fights["Fight_Date"] = pd.to_datetime(fights["Fight_Date"])


In [2]:
fights = fights[fights["Fight_Result"].isin(["Victory", "Loss"])]
fights["Fight_Result"] = fights["Fight_Result"].apply(lambda x: 1 if x == "Victory" else 0)

In [3]:
athletes_columns = [
    "Athlete_ID",
    "Wins",
    "Losses",
    "Significant_Strikes_Ratio",
    "Takedowns_Ratio",
    "Fighting_Category",
    "Age",
    "Height",
    "Weight",
    "Reach"
]

athletes_query = """SELECT Athlete_ID,
                         Wins,
                         Losses,
                         Significant_Strikes_Ratio,
                         Takedowns_Ratio,
                         Fighting_Category,
                         Age,
                         Height,
                         Weight,
                         Reach 
                  FROM ufc_athletes"""

athletes = pd.DataFrame(client.execute(athletes_query), columns=athletes_columns)

In [4]:
dataprep = pd.merge(fights, athletes, on='Athlete_ID', how='left')
dataprep = pd.merge(dataprep, athletes, left_on='Opponent_ID', right_on='Athlete_ID', how='left')

In [5]:
def create_fight_key(row):
    fighters = sorted([row['Athlete_ID_x'], row['Opponent_ID']])
    return f"{fighters[0]}_{fighters[1]}_{row['Fight_Date']}"

dataprep['Fight_Key'] = dataprep.apply(create_fight_key, axis=1)
dataprep = dataprep.drop_duplicates(subset=['Fight_Key'])
dataprep = dataprep.drop(columns=['Fight_Key'])

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline  # Note: Using imblearn's Pipeline
from imblearn.under_sampling import RandomUnderSampler

# Generate differences
numeric_features = ['Age', 'Height', 'Weight', 'Reach', 
                    'Significant_Strikes_Ratio', 'Takedowns_Ratio']
for feat in numeric_features:
    dataprep[f'{feat}_diff'] = dataprep[f'{feat}_x'] - dataprep[f'{feat}_y']

# Split data
X = dataprep.drop(columns=['Fight_Result'])
y = dataprep['Fight_Result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num_std', StandardScaler(), [f'{feat}_diff' for feat in numeric_features]),
        ('num_norm', MinMaxScaler(), ['Significant_Strikes_Ratio_x', 'Takedowns_Ratio_x',
                                     'Significant_Strikes_Ratio_y', 'Takedowns_Ratio_y']),
        ('cat', OneHotEncoder(), ['Fighting_Category_x', 'Fighting_Category_y'])
    ])

# Use imblearn's Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('model', XGBClassifier(n_estimators=100, random_state=42, max_depth=3, learning_rate=0.1))
])

# Train and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.66      0.64       558
           1       0.63      0.60      0.61       534

    accuracy                           0.63      1092
   macro avg       0.63      0.63      0.63      1092
weighted avg       0.63      0.63      0.63      1092


Confusion Matrix:
[[367 191]
 [213 321]]


In [ ]:
y_pred = pipeline.predict([[]])

array([1, 1, 0, ..., 0, 1, 0], shape=(1092,))

In [9]:
X_test

,Athlete_ID_x,Opponent_ID,Fight_Date,Delta_Time,Wins_x,Losses_x,Significant_Strikes_Ratio_x,Takedowns_Ratio_x,Fighting_Category_x,Age_x,...,Age_y,Height_y,Weight_y,Reach_y,Age_diff,Height_diff,Weight_diff,Reach_diff,Significant_Strikes_Ratio_diff,Takedowns_Ratio_diff
4326,erik-koch,shane-campbell,2016-05-29,9.0,16,6,0.000000,0.000000,Grappling,34,...,36.0,72.0,156.0,71.0,-2.0,-2.0,14.5,0.5,-0.598639,0.000000
7262,joshua-van,zhalgas-zhumagulov,2023-06-24,2.0,12,2,0.540396,0.000000,Striking,23,...,34.0,64.0,125.5,66.5,-11.0,1.0,0.5,-1.5,0.109799,-0.083333
5508,irene-aldana,karol-rosa,2023-12-17,1.0,15,8,0.399325,0.500000,Striking,36,...,30.0,65.0,135.5,67.5,6.0,4.0,0.5,1.0,-0.160413,0.500000
5304,henry-cejudo,jussier-formiga,2015-11-21,10.0,16,4,0.467763,0.246753,Grappling,37,...,38.0,65.0,126.0,67.0,-1.0,-1.0,9.0,-3.0,0.071769,0.004818
4618,francisco-trinaldo,jesse-ronson,2014-02-15,11.0,28,9,0.452466,0.320000,Grappling,44,...,38.0,70.0,155.5,70.0,6.0,-1.0,14.5,0.0,-0.024993,0.034286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,brandon-vera,jon-jones,2010-03-21,15.0,12,7,0.000000,0.000000,Grappling,45,...,37.0,76.0,237.6,84.5,8.0,-2.0,3.4,-84.5,-0.589077,-0.367347
1940,carlo-prater,erick-silva,2012-01-14,13.0,30,12,0.470199,0.000000,Striking,41,...,38.0,71.0,170.5,74.0,3.0,0.0,-14.5,-74.0,0.470199,0.000000
8113,laureano-staropoli,thiago-alves,2019-11-05,6.0,9,5,0.340183,0.000000,Striking,30,...,39.0,69.0,171.0,70.0,-9.0,4.0,14.5,1.5,0.340183,0.000000
4139,edson-barboza,giga-chikadze,2021-08-28,4.0,24,12,0.441220,0.318182,Striking,39,...,36.0,72.0,146.0,74.0,3.0,-1.0,-0.5,1.0,0.016887,0.151515


In [11]:
athletes[athletes["Athlete_ID"].str.contains("alex")]

,Athlete_ID,Wins,Losses,Significant_Strikes_Ratio,Takedowns_Ratio,Fighting_Category,Age,Height,Weight,Reach
79,alex-andrade,0,0,0.000000,0.000000,Grappling,49,0.0,0.0,0.0
80,alex-caceres,21,15,0.499671,0.473684,Striking,35,70.0,158.8,73.5
81,alex-chambers,5,4,0.540000,0.411765,Striking,44,63.0,115.8,63.0
82,alex-da-silva,21,4,0.463415,0.250000,Striking,27,68.0,155.5,73.0
83,alex-enlund,14,2,0.000000,0.000000,Grappling,36,70.0,145.0,73.0
84,alex-garcia,15,5,0.313609,0.509091,Grappling,35,69.0,170.5,72.0
85,alex-gorgees,7,1,0.322581,0.000000,Striking,0,0.0,0.0,0.0
86,alex-hunter,0,0,0.000000,0.000000,Grappling,0,0.0,0.0,0.0
87,alex-karalexis,8,2,0.000000,0.000000,Grappling,0,171.0,0.0,0.0
88,alex-morono,24,11,0.402570,0.000000,Striking,34,71.0,170.5,72.0
